In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pefile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 6.1 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import numpy
import pickle
import pefile
import sklearn.ensemble as ek
from sklearn.model_selection import train_test_split
from sklearn import tree, linear_model
from sklearn.feature_selection import SelectFromModel
import joblib
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn import svm
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/SEM6_Project/Malicious_File/PE_Malicious_File_Detection/malware_data.csv',sep='|')
df.groupby(df['legitimate']).size()

,0
legitimate,
0,96724
1,41323


In [ ]:
df.head()

,Name,md5,Machine,SizeOfOptionalHeader,Characteristics,MajorLinkerVersion,MinorLinkerVersion,SizeOfCode,SizeOfInitializedData,SizeOfUninitializedData,...,ResourcesNb,ResourcesMeanEntropy,ResourcesMinEntropy,ResourcesMaxEntropy,ResourcesMeanSize,ResourcesMinSize,ResourcesMaxSize,LoadConfigurationSize,VersionInformationSize,legitimate
0,memtest.exe,631ea355665f28d4707448e442fbf5b8,332,224,258,9,0,361984,115712,0,...,4,3.262823,2.568844,3.537939,8797.000000,216,18032,0,16,1
1,ose.exe,9d10f99a6712e28f8acd5641e3a7ea6b,332,224,3330,9,0,130560,19968,0,...,2,4.250461,3.420744,5.080177,837.000000,518,1156,72,18,1
2,setup.exe,4d92f518527353c0db88a70fddcfd390,332,224,3330,9,0,517120,621568,0,...,11,4.426324,2.846449,5.271813,31102.272727,104,270376,72,18,1
3,DW20.EXE,a41e524f8d45f0074fd07805ff0c9b12,332,224,258,9,0,585728,369152,0,...,10,4.364291,2.669314,6.400720,1457.000000,90,4264,72,18,1
4,dwtrig20.exe,c87e561258f2f8650cef999bf643a731,332,224,258,9,0,294912,247296,0,...,2,4.306100,3.421598,5.190603,1074.500000,849,1300,72,18,1


In [ ]:
df.columns

Index(['Name', 'md5', 'Machine', 'SizeOfOptionalHeader', 'Characteristics',
       'MajorLinkerVersion', 'MinorLinkerVersion', 'SizeOfCode',
       'SizeOfInitializedData', 'SizeOfUninitializedData',
       'AddressOfEntryPoint', 'BaseOfCode', 'BaseOfData', 'ImageBase',
       'SectionAlignment', 'FileAlignment', 'MajorOperatingSystemVersion',
       'MinorOperatingSystemVersion', 'MajorImageVersion', 'MinorImageVersion',
       'MajorSubsystemVersion', 'MinorSubsystemVersion', 'SizeOfImage',
       'SizeOfHeaders', 'CheckSum', 'Subsystem', 'DllCharacteristics',
       'SizeOfStackReserve', 'SizeOfStackCommit', 'SizeOfHeapReserve',
       'SizeOfHeapCommit', 'LoaderFlags', 'NumberOfRvaAndSizes', 'SectionsNb',
       'SectionsMeanEntropy', 'SectionsMinEntropy', 'SectionsMaxEntropy',
       'SectionsMeanRawsize', 'SectionsMinRawsize', 'SectionMaxRawsize',
       'SectionsMeanVirtualsize', 'SectionsMinVirtualsize',
       'SectionMaxVirtualsize', 'ImportsNbDLL', 'ImportsNb',
       'Impor

In [ ]:
print(df.isnull().sum())

Name                           0
md5                            0
Machine                        0
SizeOfOptionalHeader           0
Characteristics                0
MajorLinkerVersion             0
MinorLinkerVersion             0
SizeOfCode                     0
SizeOfInitializedData          0
SizeOfUninitializedData        0
AddressOfEntryPoint            0
BaseOfCode                     0
BaseOfData                     0
ImageBase                      0
SectionAlignment               0
FileAlignment                  0
MajorOperatingSystemVersion    0
MinorOperatingSystemVersion    0
MajorImageVersion              0
MinorImageVersion              0
MajorSubsystemVersion          0
MinorSubsystemVersion          0
SizeOfImage                    0
SizeOfHeaders                  0
CheckSum                       0
Subsystem                      0
DllCharacteristics             0
SizeOfStackReserve             0
SizeOfStackCommit              0
SizeOfHeapReserve              0
SizeOfHeap

In [ ]:
print(df.dtypes)

Name                            object
md5                             object
Machine                          int64
SizeOfOptionalHeader             int64
Characteristics                  int64
MajorLinkerVersion               int64
MinorLinkerVersion               int64
SizeOfCode                       int64
SizeOfInitializedData            int64
SizeOfUninitializedData          int64
AddressOfEntryPoint              int64
BaseOfCode                       int64
BaseOfData                       int64
ImageBase                      float64
SectionAlignment                 int64
FileAlignment                    int64
MajorOperatingSystemVersion      int64
MinorOperatingSystemVersion      int64
MajorImageVersion                int64
MinorImageVersion                int64
MajorSubsystemVersion            int64
MinorSubsystemVersion            int64
SizeOfImage                      int64
SizeOfHeaders                    int64
CheckSum                         int64
Subsystem                

In [ ]:
# Exclude Name, md5, and target
X = df.drop(columns=["Name", "md5", "legitimate"])
y = df["legitimate"]

In [ ]:
# Scale the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Input dimension
input_dim = X.shape[1]

# Autoencoder architecture
input_layer = Input(shape=(input_dim,))
encoder = Dense(64, activation='relu', kernel_regularizer=l2(0.001))(input_layer)
encoder = BatchNormalization()(encoder)
encoder = Dropout(0.3)(encoder)  # Dropout layer
encoder = Dense(32, activation='relu', kernel_regularizer=l2(0.001))(encoder)
encoder = BatchNormalization()(encoder)
encoder = Dropout(0.3)(encoder)  # Dropout layer
bottleneck = Dense(16, activation='relu', kernel_regularizer=l2(0.001), name='bottleneck')(encoder)  # Bottleneck layer

decoder = Dense(32, activation='relu', kernel_regularizer=l2(0.001))(bottleneck)
decoder = BatchNormalization()(decoder)
decoder = Dropout(0.3)(decoder)  # Dropout layer
decoder = Dense(64, activation='relu', kernel_regularizer=l2(0.001))(decoder)
decoder = BatchNormalization()(decoder)
output_layer = Dense(input_dim, activation='linear')(decoder)

# Build and compile the model
autoencoder = Model(inputs=input_layer, outputs=output_layer)
autoencoder.compile(optimizer='adam', loss='mse')

# Add noise to the data
X_noisy = X + np.random.normal(0, 0.01, X.shape)

# Callbacks: Early Stopping and Learning Rate Scheduler
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6
)

# Train the autoencoder
history = autoencoder.fit(
    X_noisy, X,  # Train on noisy data, reconstruct clean data
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    verbose=1,
    callbacks=[early_stopping, lr_scheduler]
)

Epoch 1/100
3452/3452 ━━━━━━━━━━━━━━━━━━━━ 27s 5ms/step - loss: 0.9464 - val_loss: 0.8448 - learning_rate: 0.0010
Epoch 2/100
3452/3452 ━━━━━━━━━━━━━━━━━━━━ 31s 3ms/step - loss: 0.7031 - val_loss: 0.8243 - learning_rate: 0.0010
Epoch 3/100
3452/3452 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 0.6541 - val_loss: 0.8068 - learning_rate: 0.0010
Epoch 4/100
3452/3452 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - loss: 0.6550 - val_loss: 0.8229 - learning_rate: 0.0010
Epoch 5/100
3452/3452 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.7042 - val_loss: 0.8831 - learning_rate: 0.0010
Epoch 6/100
3452/3452 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.6342 - val_loss: 0.8431 - learning_rate: 0.0010
Epoch 7/100
3452/3452 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.6761 - val_loss: 0.7979 - learning_rate: 5.0000e-04
Epoch 8/100
3452/3452 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 0.6163 - val_loss: 0.8280 - learning_rate: 5.0000e-04
Epoch 9/100
3452/3452 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.5219 - val_loss

In [ ]:
autoencoder.save('autoencoder_model.h5')
print("Model saved as 'autoencoder_model.h5'")

Model saved as 'autoencoder_model.h5'


In [ ]:
# Save the trained model to a file
autoencoder.save('auto_model.keras')
print("Model saved as 'auto_model.keras'")

Model saved as 'auto_model.keras'


In [ ]:
from tensorflow.keras.models import Model

# Define the bottleneck extractor model
bottleneck_model = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer("bottleneck").output)

# Generate bottleneck features for train and test sets
X_bottleneck = bottleneck_model.predict(X)

bottleneck_dim = X_bottleneck.shape[1]  # Dimensionality of the bottleneck features

4314/4314 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step


In [ ]:
print(bottleneck_dim)

16


In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_bottleneck, y, test_size=0.25, random_state=42, stratify=y
)

# Print the shapes of the splits to confirm
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (103535, 16)
X_test shape: (34512, 16)
y_train shape: (103535,)
y_test shape: (34512,)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Neural network architecture
model = Sequential([
    Input(shape=(bottleneck_dim,)),  # Define the input shape
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Change activation for multiclass tasks
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-6
)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    verbose=1,
    callbacks=[early_stopping, reduce_lr]
)


Epoch 1/100
2589/2589 ━━━━━━━━━━━━━━━━━━━━ 17s 5ms/step - accuracy: 0.9441 - loss: 0.1623 - val_accuracy: 0.9778 - val_loss: 0.0637 - learning_rate: 0.0010
Epoch 2/100
2589/2589 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.9712 - loss: 0.0870 - val_accuracy: 0.9786 - val_loss: 0.0584 - learning_rate: 0.0010
Epoch 3/100
2589/2589 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.9739 - loss: 0.0805 - val_accuracy: 0.9797 - val_loss: 0.0565 - learning_rate: 0.0010
Epoch 4/100
2589/2589 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9749 - loss: 0.0746 - val_accuracy: 0.9797 - val_loss: 0.0552 - learning_rate: 0.0010
Epoch 5/100
2589/2589 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.9762 - loss: 0.0712 - val_accuracy: 0.9803 - val_loss: 0.0553 - learning_rate: 0.0010
Epoch 6/100
2589/2589 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.9775 - loss: 0.0678 - val_accuracy: 0.9793 - val_loss: 0.0551 - learning_rate: 0.0010
Epoch 7/100
2589/2589 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 98.63%


In [ ]:
# Save the trained model to a file
model.save('maliciousNN_model.h5')
print("Model saved as 'trained_model.h5'")

Model saved as 'trained_model.h5'


In [ ]:
# Save the trained model to a file
model.save('maliNN_model.keras')
print("Model saved as 'maliNN_model.keras'")

Model saved as 'maliNN_model.keras'


In [ ]:
%%writefile Final_Testing.py
"""
This file extracts the required information of a given file using the PE library.
"""

import pefile
import os
import array
import math
import pickle
import sys
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model

def get_entropy(data):
    if len(data) == 0:
        return 0.0
    occurrences = array.array('L', [0] * 256)
    for x in data:
        occurrences[x if isinstance(x, int) else ord(x)] += 1
    entropy = 0
    for x in occurrences:
        if x:
            p_x = float(x) / len(data)
            entropy -= p_x * math.log(p_x, 2)
    return entropy


def get_resources(pe):
    """Extract resources: [entropy, size]"""
    resources = []
    if hasattr(pe, 'DIRECTORY_ENTRY_RESOURCE'):
        try:
            for resource_type in pe.DIRECTORY_ENTRY_RESOURCE.entries:
                if hasattr(resource_type, 'directory'):
                    for resource_id in resource_type.directory.entries:
                        if hasattr(resource_id, 'directory'):
                            for resource_lang in resource_id.directory.entries:
                                data = pe.get_data(resource_lang.data.struct.OffsetToData,
                                                   resource_lang.data.struct.Size)
                                size = resource_lang.data.struct.Size
                                entropy = get_entropy(data)

                                resources.append([entropy, size])
        except Exception as e:
            return resources
    return resources

def get_version_info(pe):
    """Return version infos"""
    res = {}
    for fileinfo in pe.FileInfo:
        if fileinfo.Key == 'StringFileInfo':
            for st in fileinfo.StringTable:
                for entry in st.entries.items():
                    res[entry[0]] = entry[1]
        if fileinfo.Key == 'VarFileInfo':
            for var in fileinfo.Var:
                res[var.entry.items()[0][0]] = var.entry.items()[0][1]
    if hasattr(pe, 'VS_FIXEDFILEINFO'):
        res['flags'] = pe.VS_FIXEDFILEINFO.FileFlags
        res['os'] = pe.VS_FIXEDFILEINFO.FileOS
        res['type'] = pe.VS_FIXEDFILEINFO.FileType
        res['file_version'] = pe.VS_FIXEDFILEINFO.FileVersionLS
        res['product_version'] = pe.VS_FIXEDFILEINFO.ProductVersionLS
        res['signature'] = pe.VS_FIXEDFILEINFO.Signature
        res['struct_version'] = pe.VS_FIXEDFILEINFO.StrucVersion
    return res

def extract_infos(fpath):
    res = {}
    pe = pefile.PE(fpath)
    res['Machine'] = pe.FILE_HEADER.Machine
    res['SizeOfOptionalHeader'] = pe.FILE_HEADER.SizeOfOptionalHeader
    res['Characteristics'] = pe.FILE_HEADER.Characteristics
    res['MajorLinkerVersion'] = pe.OPTIONAL_HEADER.MajorLinkerVersion
    res['MinorLinkerVersion'] = pe.OPTIONAL_HEADER.MinorLinkerVersion
    res['SizeOfCode'] = pe.OPTIONAL_HEADER.SizeOfCode
    res['SizeOfInitializedData'] = pe.OPTIONAL_HEADER.SizeOfInitializedData
    res['SizeOfUninitializedData'] = pe.OPTIONAL_HEADER.SizeOfUninitializedData
    res['AddressOfEntryPoint'] = pe.OPTIONAL_HEADER.AddressOfEntryPoint
    res['BaseOfCode'] = pe.OPTIONAL_HEADER.BaseOfCode
    try:
        res['BaseOfData'] = pe.OPTIONAL_HEADER.BaseOfData
    except AttributeError:
        res['BaseOfData'] = 0
    res['ImageBase'] = pe.OPTIONAL_HEADER.ImageBase
    res['SectionAlignment'] = pe.OPTIONAL_HEADER.SectionAlignment
    res['FileAlignment'] = pe.OPTIONAL_HEADER.FileAlignment
    res['MajorOperatingSystemVersion'] = pe.OPTIONAL_HEADER.MajorOperatingSystemVersion
    res['MinorOperatingSystemVersion'] = pe.OPTIONAL_HEADER.MinorOperatingSystemVersion
    res['MajorImageVersion'] = pe.OPTIONAL_HEADER.MajorImageVersion
    res['MinorImageVersion'] = pe.OPTIONAL_HEADER.MinorImageVersion
    res['MajorSubsystemVersion'] = pe.OPTIONAL_HEADER.MajorSubsystemVersion
    res['MinorSubsystemVersion'] = pe.OPTIONAL_HEADER.MinorSubsystemVersion
    res['SizeOfImage'] = pe.OPTIONAL_HEADER.SizeOfImage
    res['SizeOfHeaders'] = pe.OPTIONAL_HEADER.SizeOfHeaders
    res['CheckSum'] = pe.OPTIONAL_HEADER.CheckSum
    res['Subsystem'] = pe.OPTIONAL_HEADER.Subsystem
    res['DllCharacteristics'] = pe.OPTIONAL_HEADER.DllCharacteristics
    res['SizeOfStackReserve'] = pe.OPTIONAL_HEADER.SizeOfStackReserve
    res['SizeOfStackCommit'] = pe.OPTIONAL_HEADER.SizeOfStackCommit
    res['SizeOfHeapReserve'] = pe.OPTIONAL_HEADER.SizeOfHeapReserve
    res['SizeOfHeapCommit'] = pe.OPTIONAL_HEADER.SizeOfHeapCommit
    res['LoaderFlags'] = pe.OPTIONAL_HEADER.LoaderFlags
    res['NumberOfRvaAndSizes'] = pe.OPTIONAL_HEADER.NumberOfRvaAndSizes

    # Sections
    res['SectionsNb'] = len(pe.sections)
    entropy = list(map(lambda x: x.get_entropy(), pe.sections))
    res['SectionsMeanEntropy'] = sum(entropy)/float(len(entropy))
    res['SectionsMinEntropy'] = min(entropy)
    res['SectionsMaxEntropy'] = max(entropy)
    raw_sizes = list(map(lambda x: x.SizeOfRawData, pe.sections))
    res['SectionsMeanRawsize'] = sum(raw_sizes)/float(len(raw_sizes))
    res['SectionsMinRawsize'] = min(raw_sizes)
    res['SectionsMaxRawsize'] = max(raw_sizes)
    virtual_sizes = list(map(lambda x: x.Misc_VirtualSize, pe.sections))
    res['SectionsMeanVirtualsize'] = sum(virtual_sizes)/float(len(virtual_sizes))
    res['SectionsMinVirtualsize'] = min(virtual_sizes)
    res['SectionMaxVirtualsize'] = max(virtual_sizes)

    # Imports
    try:
        res['ImportsNbDLL'] = len(pe.DIRECTORY_ENTRY_IMPORT)
        imports = sum([x.imports for x in pe.DIRECTORY_ENTRY_IMPORT], [])
        res['ImportsNb'] = len(imports)
        res['ImportsNbOrdinal'] = len(list(filter(lambda x: x.name is None, imports)))
    except AttributeError:
        res['ImportsNbDLL'] = 0
        res['ImportsNb'] = 0
        res['ImportsNbOrdinal'] = 0

    # Exports
    try:
        res['ExportNb'] = len(pe.DIRECTORY_ENTRY_EXPORT.symbols)
    except AttributeError:
        # No export
        res['ExportNb'] = 0

    # Resources
    resources = get_resources(pe)
    res['ResourcesNb'] = len(resources)
    if len(resources) > 0:
        entropy = list(map(lambda x: x[0], resources))
        res['ResourcesMeanEntropy'] = sum(entropy)/float(len(entropy))
        res['ResourcesMinEntropy'] = min(entropy)
        res['ResourcesMaxEntropy'] = max(entropy)
        sizes = list(map(lambda x: x[1], resources))
        res['ResourcesMeanSize'] = sum(sizes)/float(len(sizes))
        res['ResourcesMinSize'] = min(sizes)
        res['ResourcesMaxSize'] = max(sizes)
    else:
        res['ResourcesNb'] = 0
        res['ResourcesMeanEntropy'] = 0
        res['ResourcesMinEntropy'] = 0
        res['ResourcesMaxEntropy'] = 0
        res['ResourcesMeanSize'] = 0
        res['ResourcesMinSize'] = 0
        res['ResourcesMaxSize'] = 0

    # Load configuration size
    try:
        res['LoadConfigurationSize'] = pe.DIRECTORY_ENTRY_LOAD_CONFIG.struct.Size
    except AttributeError:
        res['LoadConfigurationSize'] = 0

    # Version configuration size
    try:
        version_infos = get_version_info(pe)
        res['VersionInformationSize'] = len(version_infos.keys())
    except AttributeError:
        res['VersionInformationSize'] = 0

    return res

if __name__ == '__main__':
    # Load the autoencoder encoder and ANN models
    autoencoder = load_model('auto_model.keras')  # Autoencoder model
    ann_model = load_model('maliNN_model.keras')  # Trained neural network model

    # Extract information from the PE file
    data = extract_infos(sys.argv[1])

    # Prepare the features for the neural network
    pe_features = [value for value in data.values()]
    pe_features = np.array(pe_features).reshape(1, -1)

    # Pass the features through the autoencoder encoder to extract reduced features
    encoder_model = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('bottleneck').output)
    X_bottleneck = encoder_model.predict(pe_features)

    # Make predictions using the ANN model
    prediction = ann_model.predict(X_bottleneck)

    # Print the prediction result
    if prediction > 0.5:
        print(f"The file {os.path.basename(sys.argv[1])} is malicious.")
    else:
        print(f"The file {os.path.basename(sys.argv[1])} is legitimate.")

Overwriting Final_Testing.py


In [ ]:
%run Final_Testing.py /content/drive/MyDrive/SEM6_Project/Malicious_File/PE_Malicious_File_Detection/Test_Sample/sublime_text_build_4169_x64_setup.exe

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 295ms/step
The file sublime_text_build_4169_x64_setup.exe is legitimate.


In [ ]:
!git clone https://github.com/iosifache/DikeDataset.git

Cloning into 'DikeDataset'...
remote: Enumerating objects: 11989, done.
remote: Counting objects: 100% (11989/11989), done.
remote: Compressing objects: 100% (9243/9243), done.
remote: Total 11989 (delta 2749), reused 11981 (delta 2741), pack-reused 0 (from 0)
Receiving objects: 100% (11989/11989), 2.38 GiB | 17.49 MiB/s, done.
Resolving deltas: 100% (2749/2749), done.
Updating files: 100% (11936/11936), done.


## Legitimate File Prediction

In [ ]:
%run Final_Testing.py /content/drive/MyDrive/SEM6_Project/Malicious_File/PE_Malicious_File_Detection/Test_Sample/sublime_text_build_4169_x64_setup.exe

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step
The file sublime_text_build_4169_x64_setup.exe is legitimate.


In [ ]:
%run Final_Testing.py /content/DikeDataset/files/benign/08805d42f83cab69e6b93d9717da43d4d61892942b2573e45ec0e97adc6b506e.exe

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 277ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 297ms/step
The file 08805d42f83cab69e6b93d9717da43d4d61892942b2573e45ec0e97adc6b506e.exe is legitimate.


## Malicious File Prediction

In [ ]:
%run Final_Testing.py /content/DikeDataset/files/malware/005d62ccb914fe69ed4795a68a2ec0c679e4713c50a73d90524b7bac1240fefd.exe

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 360ms/step
The file 005d62ccb914fe69ed4795a68a2ec0c679e4713c50a73d90524b7bac1240fefd.exe is malicious.


In [ ]:
%run Final_Testing.py /content/DikeDataset/files/malware/0078e3961d934fcf2782d77812d2ba7177e23207099333a0cd084682ca0ef3ff.exe

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 530ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 746ms/step
The file 0078e3961d934fcf2782d77812d2ba7177e23207099333a0cd084682ca0ef3ff.exe is malicious.
